In [1]:
import pandas as pd

In [2]:
schoolfile = 'Resources/schools_complete.csv'
studentfile = 'Resources/students_complete.csv'

csv_school = pd.read_csv(schoolfile)
csv_student = pd.read_csv(studentfile)
csv_school.head()
csv_student.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [3]:
## District Summary

# * Create a high level snapshot (in table form) of the district's key metrics, including:
#   * Total Schools
#   * Total Students
#   * Total Budget
#   * Average Math Score
#   * Average Reading Score
#   * % Passing Math
#   * % Passing Reading
#   * Overall Passing Rate (Average of the above two)
totalschool = csv_school['School ID'].count()
totalstudent = csv_student['Student ID'].count()
totalbudge = csv_school['budget'].sum()
avgmath = csv_student['math_score'].mean()
avgreading = csv_student['reading_score'].mean()
passmath = csv_student.loc[csv_student['math_score']>=70 ,'Student ID'].count()/totalstudent
passreadig = csv_student.loc[csv_student['reading_score']>=70,'Student ID'].count()/totalstudent
passrate = (passmath + passreadig)/2

DistrictSummary = pd.DataFrame({'Total Schools':totalschool
                               ,'Total Students':totalstudent
                               ,'Total Budget':totalbudge
                               ,'Average Math Score':avgmath
                               ,'verage Reading Score':avgreading
                               ,'% Passing Math':passmath
                               ,'% Passing Reading':passreadig
                               ,'Overall Passing Rate':passrate}, index=[0])


DistrictSummary = DistrictSummary.style.format({'Total Budget': '${:,.2f}'
                                                ,'% Passing Math':'{:.2%}'
                                               ,'% Passing Reading':'{:.2%}'
                                               ,'Overall Passing Rate':'{:.2%}'})
DistrictSummary

,Total Schools,Total Students,Total Budget,Average Math Score,verage Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428.00",78.9854,81.8778,74.98%,85.81%,80.39%


In [4]:
# ### School Summary

# * Create an overview table that summarizes key metrics about each school, including:
#   * School Name
#   * School Type
#   * Total Students
#   * Total School Budget
#   * Per Student Budget
#   * Average Math Score
#   * Average Reading Score
#   * % Passing Math
#   * % Passing Reading
#   * Overall Passing Rate (Average of the above two)


In [5]:
df = pd.merge(csv_student,csv_school, how = 'left', on ='school_name')

group_school = df.groupby(['school_name','type'])

schoolstudent = group_school['Student ID'].count()
schoolbudget = group_school['budget'].mean()
schoolperbudget = schoolbudget/schoolstudent
schoolavgmath = group_school['math_score'].mean()
schoolavgread = group_school['reading_score'].mean()
schoolpassmath = group_school['math_score'].agg(lambda x : (x>=70).sum()/len(x))
schoolpassread = group_school['reading_score'].agg(lambda x:(x>=70).sum()/len(x))
schooloverallpass = (schoolpassmath+schoolpassread)/2

schoolsummary =pd.DataFrame({'Total Students':schoolstudent
                            ,'Total School Budget':schoolbudget
                            ,'Per Student Budget':schoolperbudget
                            ,'Average Math Score':schoolavgmath
                            ,'Average Reading Score':schoolavgread
                            ,'% Passing Math': schoolpassmath
                            ,'% Passing Reading':schoolpassread
                            ,'Overall Passing Rate':schooloverallpass}).reset_index()

schoolsummary_format = schoolsummary.style.format({'Total Students':'{:,}'
                                           ,'Total School Budget':'${:,.2f}'
                                           ,'Per Student Budget':'${:,.2f}'
                                           ,'% Passing Math':'{:.2%}'
                                           ,'% Passing Reading':'{:.2%}'
                                           ,'Overall Passing Rate':'{:.2%}'})
schoolsummary_format

,school_name,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0484,81.034,66.68%,81.93%,74.31%
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.0619,83.9758,94.13%,97.04%,95.59%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7118,81.158,65.99%,80.74%,73.36%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1026,80.7463,68.31%,79.30%,73.80%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.3515,83.8168,93.39%,97.14%,95.27%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.2898,80.9344,66.75%,80.86%,73.81%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.8033,83.815,92.51%,96.25%,94.38%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6294,81.1827,65.68%,81.32%,73.50%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.0725,80.9664,66.06%,81.22%,73.64%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.59%,95.95%,95.27%


In [6]:
### Top Performing Schools (By Passing Rate)
top5_school = schoolsummary.nlargest(5,'Overall Passing Rate').reset_index(drop= True)
top5_school_format = top5_school.style.format({'Total Students':'{:,}'
                                           ,'Total School Budget':'${:,.2f}'
                                           ,'Per Student Budget':'${:,.2f}'
                                           ,'% Passing Math':'{:.2%}'
                                           ,'% Passing Reading':'{:.2%}'
                                           ,'Overall Passing Rate':'{:.2%}'})
top5_school_format

,school_name,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.0619,83.9758,94.13%,97.04%,95.59%
1,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.4183,83.8489,93.27%,97.31%,95.29%
2,Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.59%,95.95%,95.27%
3,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.3515,83.8168,93.39%,97.14%,95.27%
4,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.2742,83.9895,93.87%,96.54%,95.20%


In [7]:
### Bottom Performing Schools (By Passing Rate)
bottom5_school = schoolsummary.nsmallest(5,'Overall Passing Rate').reset_index(drop= True)
bottom5_school_format = bottom5_school.style.format({'Total Students':'{:,}'
                                           ,'Total School Budget':'${:,.2f}'
                                           ,'Per Student Budget':'${:,.2f}'
                                           ,'% Passing Math':'{:.2%}'
                                           ,'% Passing Reading':'{:.2%}'
                                           ,'Overall Passing Rate':'{:.2%}'})
bottom5_school_format

,school_name,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.8427,80.7447,66.37%,80.22%,73.29%
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7118,81.158,65.99%,80.74%,73.36%
2,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6294,81.1827,65.68%,81.32%,73.50%
3,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.0725,80.9664,66.06%,81.22%,73.64%
4,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1026,80.7463,68.31%,79.30%,73.80%


In [8]:
### Math Scores by Grade\*\*

grademath = pd.pivot_table(df,index='school_name', columns='grade', values='math_score')

grademath = grademath[['9th','10th','11th','12th']]
grademath

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [9]:
### Reading Scores by Grade

graderead = pd.pivot_table(df, index='school_name',columns='grade',values = 'reading_score')

graderead = graderead[['9th','10th','11th','12th']]
graderead

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [10]:
### Scores by School Spending
# * Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
#   * Average Math Score
#   * Average Reading Score
#   * % Passing Math
#   * % Passing Reading
#   * Overall Passing Rate (Average of the above two)

# (schoolsummary['Per Student Budget'].max() - schoolsummary['Per Student Budget'].min())/4
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

schoolsummary['Budget Level'] = pd.cut(schoolsummary['Per Student Budget'],spending_bins,labels = group_names)

group_spend = schoolsummary.groupby('Budget Level')
spendsummary = group_spend.mean().iloc[:,3:]

spendsummary_format = spendsummary.style.format({'% Passing Math':'{:.2%}'
                                                ,'% Passing Reading':'{:.2%}'
                                                ,'Overall Passing Rate':'{:.2%}'})
spendsummary_format

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Budget Level,,,,,
<$585,83.4554,83.9338,93.46%,96.61%,95.04%
$585-615,83.5997,83.8852,94.23%,95.90%,95.07%
$615-645,79.0792,81.8914,75.67%,86.11%,80.89%
$645-675,76.9972,81.0278,66.16%,81.13%,73.65%


In [11]:
### Scores by School Size

size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

schoolsummary['School Size'] = pd.cut(schoolsummary['Total Students'],size_bins,labels=group_names)
size_group = schoolsummary.groupby('School Size')

sizesummary = size_group.mean().iloc[:,3:]

sizesummary_format = sizesummary.style.format({'% Passing Math':'{:.2%}'
                                               ,'% Passing Reading':'{:.2%}'
                                               ,'Overall Passing Rate':'{:.2%}'})
sizesummary_format

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.8216,83.9298,93.55%,96.10%,94.82%
Medium (1000-2000),83.3747,83.8644,93.60%,96.79%,95.20%
Large (2000-5000),77.7464,81.3445,69.96%,82.77%,76.36%


In [12]:
### Scores by School Type

type_group = schoolsummary.groupby('type')
typesummary = type_group.mean().iloc[:,3:]
typesummary_format = typesummary.style.format({'% Passing Math':'{:.2%}'
                                               ,'% Passing Reading':'{:.2%}'
                                               ,'Overall Passing Rate':'{:.2%}'})
typesummary_format

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.4739,83.8964,93.62%,96.59%,95.10%
District,76.9567,80.9666,66.55%,80.80%,73.67%
